In [1]:
import numpy as np
#from scipy import stats
import matplotlib.pyplot as plt
import matplotlib

import torch 
from torch import distributions
from torch import nn
from torch.utils import data

In [ ]:
class DoubleWell(object):

    params_default = {'a4' : 1.0,
                      'a2' : 6.0,
                      'a1' : 1.0,
                      'k' : 1.0,
                      'dim' : 1}

    def __init__(self, params=None):
        # set parameters
        if params is None:
            params = self.__class__.params_default
        self.params = params

        # useful variables
        self.dim = self.params['dim']


    def energy(self, x):
        dimer_energy =self.params['a4'] * x[:, 0] ** 4 - self.params['a2'] * x[:, 0] ** 2 + self.params['a1'] * x[:, 0]
        oscillator_energy = 0.0
        if self.dim == 2:
            oscillator_energy = (self.params['k'] / 2.0) * x[:, 1] ** 2
        if self.dim > 2:
            oscillator_energy = np.sum((self.params['k'] / 2.0) * x[:, 1:] ** 2, axis=1)
        return  dimer_energy + oscillator_energy

    def energy_tf(self, x):
        return self.energy(x)

    def plot_dimer_energy(self, axis=None, temperature=1.0):
        """ Plots the dimer energy to the standard figure """
        x_grid = np.linspace(-3, 3, num=200)
        if self.dim == 1:
            X = x_grid[:, None]
        else:
            X = np.hstack([x_grid[:, None], np.zeros((x_grid.size, self.dim - 1))])
        energies = self.energy(X) / temperature

        import matplotlib.pyplot as plt
        if axis is None:
            axis = plt.gca()
        #plt.figure(figsize=(5, 4))
        axis.plot(x_grid, energies, linewidth=3, color='black')
        axis.set_xlabel('x / a.u.')
        axis.set_ylabel('Energy / kT')
        axis.set_ylim(energies.min() - 2.0, energies[int(energies.size / 2)] + 2.0)

        return x_grid, energies

In [2]:
def E( x, a=1, b=-6, c=0):
  return a*x**4+b*x**2+c*x

In [3]:
def plot_energy(x):
  counts, bins = np.histogram(x, bins = 200 )
  anchors = (bins[1:] + bins[:-1]) / 2
  probs = counts / np.sum(counts)

  anchors = anchors[np.where(probs > 0.0001)]
  probs = probs[np.where(probs > 0.0001)]

  f = -np.log(probs)
  fn = f - np.min(f)
  x_mesh = np.linspace(-2.5,2.5,1000)
  E_mesh = E(x_mesh)
  E_mesh=E_mesh-E_mesh.min()
  plt.scatter(anchors, fn) 

  plt.xlabel("$x_1$")
  plt.ylabel(r"$(f - f_0) / k_B T$")

  plt.plot(x_mesh,E_mesh)

  plt.show()
  return 0

In [ ]:
class ParticleFilter(object):
    def __init__(self, bg, X0, capacity, optimizer=None, lr=0.001, batch_size=1024, 
                 high_energy=100, max_energy=1e10, std=1.0, w_KL=1.0, w_ML=1.0, w_RC=0.0, 
                 rc_func=None, rc_min=0.0, rc_max=1.0,
                 weigh_ML=True, mapper=None):
        """
        Parameters:
        -----------
        X0 : array or None
            If none, the Boltzmann Generator will be used to generate samples to fill the buffer. 
            If given, the buffer will be filled with random samples from X0.
        """
        self.bg = bg
        self.lr = lr
        self.batch_size = batch_size
        self.high_energy = high_energy
        self.max_energy = max_energy
        self.std = std
        self.temperature = 1.0
        self.weighML = weigh_ML
        self.mapper = mapper
        self.rc_func = rc_func

        inputs = [bg.input_x, bg.input_z]
        outputs = [bg.output_z, bg.output_x]        
        if weigh_ML:
            losses = [self.loss_ML_weighted, self.loss_KL]
        else:
            losses = [self.loss_ML, self.loss_KL]
        loss_weights = [w_ML, w_KL]
        if w_RC > 0.0:
            self.gmeans = np.linspace(rc_min, rc_max, 11)
            self.gstd = (rc_max - rc_min) / 11.0
            outputs.append(bg.output_x)
            losses.append(self.loss_RC)
            loss_weights.append(w_RC)

        # initial data processing
        self.I = np.arange(capacity)
        if X0 is None:
            _, self.X, _, _, _ = bg.sample(temperature=self.temperature, nsample=capacity)
        else:
            I_X0 = np.arange(X0.shape[0])
            Isel = np.random.choice(I_X0, size=capacity, replace=True)
            self.X = X0[Isel]

        # build estimator
        if optimizer is None:
            optimizer = keras.optimizers.adam(lr=lr)

        # assemble model
        self.dual_model = keras.models.Model(inputs=inputs, outputs=outputs)
        self.dual_model.compile(optimizer=optimizer, loss=losses, loss_weights=loss_weights)

        # training loop
        dummy_output = np.zeros((batch_size, bg.dim))
        self.y = [dummy_output for o in outputs]

        self.loss_train = []
        self.acceptance_rate = []
        self.stepsize = []
        
    def loss_ML(self, y_true, y_pred):
        z = self.bg.output_z
        Jxz = self.bg.log_det_Jxz[:, 0]
        LL = Jxz - (0.5 / (self.std**2)) * tf.reduce_sum(z**2, axis=1)
        return -LL

    def loss_ML_weighted(self, y_true, y_pred):
        from deep_boltzmann.util import linlogcut
        x = self.bg.input_x
        z = self.bg.output_z
        Jxz = self.bg.log_det_Jxz[:, 0]
        LL = Jxz - (0.5 / (self.std**2)) * tf.reduce_sum(z**2, axis=1)
        # compute energies
        E = self.bg.energy_model.energy_tf(x) / self.temperature
        Ereg = linlogcut(E, self.high_energy, self.max_energy, tf=True)
        # weights
        Ez = tf.reduce_sum(z**2, axis=1)/(2.0*self.temperature)
        logW = -Ereg + Ez - Jxz
        logW = logW - tf.reduce_max(logW)
        weights = tf.exp(logW)
        # weighted ML
        weighted_negLL = -self.batch_size * (weights * LL) / tf.reduce_sum(weights)
        return weighted_negLL

    def loss_KL(self, y_true, y_pred):
        return self.bg.log_KL_x(self.high_energy, self.max_energy, temperature_factors=self.temperature, explore=1.0)

    def train(self, epochs=2000, stepsize=1.0, verbose=1):
        """
        Parameters
        ----------
        stepsize : float or None
            MCMC stepsize (in latent space, so 1 is a large step).
            If None, uses adaptive stepsize between 0.001 and 1 depending on the acceptance rate.
        """
        if stepsize is None:  # initialize stepsize when called for the first time
            if len(self.stepsize) == 0:
                self.stepsize.append(0.1)
        else:
            self.stepsize = [stepsize]

        for e in range(epochs):
            # sample batch
            Isel = np.random.choice(self.I, size=self.batch_size, replace=True)
            x_batch = self.X[Isel]
            w_batch = np.sqrt(self.temperature) * np.random.randn(self.batch_size, self.bg.dim)
            l = self.dual_model.train_on_batch(x=[x_batch, w_batch], y=self.y)
            self.loss_train.append(l)

            # Do an MCMC step with the current BG

            # First recompute Z and logW
            z_batch, Jxz_batch = self.bg.transform_xzJ(x_batch)
            logW_old = self.bg.energy_model.energy(x_batch) / self.temperature + Jxz_batch

            # New step
            z_batch_new = z_batch + self.stepsize[-1] * np.sqrt(self.temperature) * np.random.randn(z_batch.shape[0], z_batch.shape[1])
            x_batch_new, Jzx_batch_new = self.bg.transform_zxJ(z_batch_new)
            logW_new = self.bg.energy_model.energy(x_batch_new) / self.temperature - Jzx_batch_new

            # Accept or reject according to target density
            rand = -np.log(np.random.rand(self.batch_size))
            Iacc = rand >= logW_new - logW_old

            # map accepted
            x_acc = x_batch_new[Iacc]
            if self.mapper is not None:
                x_acc = self.mapper.map(x_acc)
            self.X[Isel[Iacc]] = x_acc

            # acceptance rate
            pacc = float(np.count_nonzero(Iacc)) / float(self.batch_size)
            self.acceptance_rate.append(pacc)

            # update stepsize
            if stepsize is None:
                if len(self.acceptance_rate) > 50:  # update stepsize
                    mean_acceptance_rate = np.mean(self.acceptance_rate[-50:])
                    if mean_acceptance_rate < 0.02:
                        self.stepsize.append(max(self.stepsize[-1] * 0.98, 0.001))
                    elif mean_acceptance_rate > 0.2:
                        self.stepsize.append(min(self.stepsize[-1] * 1.02, 1.0))
                    else:
                        self.stepsize.append(self.stepsize[-1])  # just copy old stepsize
                else:
                    self.stepsize.append(self.stepsize[-1])  # just copy old stepsize

            # print
            if verbose > 0:
                str_ = 'Epoch ' + str(e) + '/' + str(epochs) + ' '
                for i in range(len(self.dual_model.metrics_names)):
                    str_ += self.dual_model.metrics_names[i] + ' '
                    str_ += '{:.4f}'.format(self.loss_train[-1][i]) + ' '
                str_ += 'p_acc ' + str(pacc) + ' '
                if stepsize is None:
                    str_ += 'step ' + str(self.stepsize[-1])
                print(str_)
                sys.stdout.flush()
